In [1]:
import os
import pandas as pd
from llama_index.experimental.query_engine import PandasQueryEngine
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.agent import ReActAgent
from llama_index.llms.groq import Groq

In [2]:
llm = Groq(model="llama3-70b-8192", api_key="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")

In [3]:
demand_df = pd.read_csv("data/demand_forecasting.csv")
demand_df

Product ID        Date  Store ID  Sales Quantity  Price Promotions  \
0           4277  2024-01-03        48             330  24.38         No   
1           5540  2024-04-29        10             334  74.98        Yes   
2           5406  2024-01-11        67             429  24.83        Yes   
3           5617  2024-04-04        17             298  13.41         No   
4           3480  2024-12-14        33             344  94.96        Yes   
...          ...         ...       ...             ...    ...        ...   
9995        6545  2024-11-26        54              47  47.12         No   
9996        4341  2024-04-08        82              59  65.11         No   
9997        8679  2024-02-16        55             481  59.97         No   
9998        6281  2024-02-15        97             142  48.70         No   
9999        4750  2024-03-19        18              19  42.43        Yes   

     Seasonality Factors    External Factors Demand Trend Customer Segments  
0               Festival  Competitor Pricing   Increasing           Regular  
1                Holiday             Weather       Stable           Premium  
2                Holiday  Economic Indicator   Decreasing           Premium  
3                    NaN  Economic Indicator       Stable           Regular  
4               Festival             Weather   Increasing           Regular  
...                  ...                 ...          ...               ...  
9995                 NaN  Competitor Pricing   Decreasing            Budget  
9996            Festival                 NaN   Increasing            Budget  
9997             Holiday  Economic Indicator       Stable           Premium  
9998            Festival                 NaN   Decreasing           Regular  
9999            Festival                 NaN   Decreasing           Regular  

[10000 rows x 10 columns]

In [41]:
from llama_index.core import PromptTemplate

instruction_str = """\
    1. Convert the query to executable Python code using Pandas.
    2. The final line of code should be a Python expression that can be called with the `eval()` function.
    3. The code should represent a solution to the query.
    4. PRINT ONLY THE EXPRESSION.
    5. Do not quote the expression."""

new_prompt = PromptTemplate(
    """\
    You are working with a pandas dataframe in Python.
    The name of the dataframe is `df`.
    This is the result of `print(df.head())`:
    {df_str}

    Follow these instructions:
    {instruction_str}
    Query: {query_str}

    Expression: """
)

context = """Use this Data if User Query is about Products Trends of Stores. This dataset contains sales transaction data of Product ID,
            Date of Product Sold, Store ID from where Product was Sold, Sales Quantity of the Product ID from that store, Price of the Product ID in that store, 
            If Promotions were applied on the Products, Seasonality Factors which affected that Products sale in that Store, 
            External Factors affecting that products sale in that store, Demand Trend of that product in that Store,
            and Customer Segments buying that Product in that Store."""

In [42]:
demand_query_engine = PandasQueryEngine( df=demand_df, verbose=True, instruction_str=instruction_str, llm=llm )

In [46]:
def Store_Trends(df):
    df['Date'] = pd.to_datetime(df['Date'])
    df['Day'] = df['Date'].dt.day
    df['Month'] = df['Date'].dt.month
    df['Year'] = df['Date'].dt.year
    df.drop(columns=['Date'], inplace=True)
    
    agg_df = df.groupby("Store ID").agg(
        Total_Products=('Product ID', 'count'),
        Total_Sales=('Sales Quantity', 'sum'),
        Avg_Price=('Price', 'mean'),
        Prod_Sold_on_Promotion=('Promotions', lambda x: (x == 'Yes').sum()),
        Products_Festival=('Seasonality Factors', lambda x: (x == 'Festival').sum()),
        Products_Holiday=('Seasonality Factors', lambda x: (x == 'Holiday').sum()),
        Competitor_Affected_Prods=('External Factors', lambda x: (x == 'Competitor Pricing').sum()),
        Weather_Affected_Prods=('External Factors', lambda x: (x == 'Weather').sum()),
        High_Demand_Prods=('Demand Trend', lambda x: (x == 'Increasing').sum()),
        Regular_Customer=('Customer Segments', lambda x: (x == 'Regular').sum()),
        Premium_Customer=('Customer Segments', lambda x: (x == 'Premium').sum()),
        Budget_Customer=('Customer Segments', lambda x: (x == 'Budget').sum()),
        Max_Sale_Day=('Day', lambda x: x.mode()[0] if not x.mode().empty else None),
        Max_Sale_Month=('Month', lambda x: x.mode()[0] if not x.mode().empty else None)).reset_index()
    
    agg_df['Avg_Sales'] = (agg_df['Total_Sales'] / agg_df['Total_Products']).round().astype(int)
    agg_df['Avg_Price'] = agg_df['Avg_Price'].round(2)
    
    return agg_df

In [43]:
instruction_str = """\
    1. Convert the query to executable Python code using Pandas.
    2. The final line of code should be a Python expression that can be called with the `eval()` function.
    3. The code should represent a solution to the query.
    4. PRINT ONLY THE EXPRESSION.
    5. Do not quote the expression."""

new_prompt = PromptTemplate(
    """\
    You are working with a pandas dataframe in Python.
    The name of the dataframe is `df`.
    This is the result of `print(df.head())`:
    {df_str}

    Follow these instructions:
    {instruction_str}
    Query: {query_str}

    Expression: """
)

context = """Purpose: The primary role of this agent is to assist users by providing accurate 
            information about Store statistics. Total Products, Average Sales, Average Price of Products, 
            Number of Products Sold With Promotions, Number of Products Sold in Festival, Number of Products Sold in Holiday,
            Number of Products affected by Competitors, Number of Products affected by Weather, Number of Products with Higher Demand, 
            Number of Regular Costumers, Number of Premium Costumers, Number of Budget Costumers, Dat of Maximum Sale, Month of Maximum Sale"""

In [47]:
store_query_engine = PandasQueryEngine(df=Store_Trends(demand_df), verbose=True, instruction_str=instruction_str, llm=llm )

In [ ]:
from llama_index.core.tools import FunctionTool
import os

note_file = os.path.join("data", "notes.txt")

def save_note(note):
    if not os.path.exists(note_file):
        open(note_file, "w")

    with open(note_file, "a") as f:
        f.writelines([note + "\n"])

    return "note saved"


note_engine = FunctionTool.from_defaults(
    fn=save_note,
    name="note_saver",
    description="this tool can save a text based note to a file for the user",
)

In [48]:
# Update your tools list
tools = [
    QueryEngineTool(
        query_engine=store_query_engine,
        metadata=ToolMetadata(
            name="store_data",
            description="""Total Products, Average Sales, Average Price of Products, 
            Number of Products Sold With Promotions, Number of Products Sold in Festival, Number of Products Sold in Holiday,
            Number of Products affected by Competitors, Number of Products affected by Weather, Number of Products with Higher Demand, 
            Number of Regular Costumers, Number of Premium Costumers, Number of Budget Costumers, Dat of Maximum Sale, Month of Maximum Sale. """,
        ),
    ),
    data_helper_tool,
    note_engine,
    QueryEngineTool(
        query_engine=demand_query_engine,
        metadata=ToolMetadata(
            name="demand_data",
            description="""This dataset contains sales transaction data with Product ID,
            Date, Store ID, Sales Quantity, Price, Promotions, Seasonality Factors, External Factors, Demand Trend,
            and Customer Segments.""",
        ),
    ),
]

In [51]:
agent = ReActAgent.from_tools(tools, llm=llm, verbose=True, context=context, memory=memory)

while (prompt := input("Enter a prompt (q to quit): ")) != "q":
    result = agent.query(prompt)

    print(result)

ReActChatFormatter.from_context is deprecated, please use `from_defaults` instead.


Enter a prompt (q to quit):  give list of most expensive products ids


> Running step 3ffe0134-dca5-4c5a-9933-046d64280502. Step input: give list of most expensive products ids
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: store_data
Action Input: {'input': 'get most expensive products'}
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
> Pandas Instructions:
```
df.loc[df['Avg_Price'].idxmax()]
```
> Pandas Output: Store ID                        53.00
Total_Products                  92.00
Total_Sales                  20213.00
Avg_Price                       60.26
Prod_Sold_on_Promotion          44.00
Products_Festival               38.00
Products_Holiday                30.00
Com

KeyboardInterrupt: 

In [40]:
QueryEngineTool?

Init signature:
QueryEngineTool(
    query_engine: llama_index.core.base.base_query_engine.BaseQueryEngine,
    metadata: llama_index.core.tools.types.ToolMetadata,
    resolve_input_errors: bool = True,
) -> None
Docstring:     
Query engine tool.

A tool making use of a query engine.

Args:
    query_engine (BaseQueryEngine): A query engine.
    metadata (ToolMetadata): The associated metadata of the query engine.
File:           c:\users\nisha\anaconda3\envs\groq1\lib\site-packages\llama_index\core\tools\query_engine.py
Type:           ABCMeta
Subclasses:     

In [6]:
from llama_index.core.tools import FunctionTool
import os

note_file = os.path.join("data", "notes.txt")

def save_note(note):
    if not os.path.exists(note_file):
        open(note_file, "w")

    with open(note_file, "a") as f:
        f.writelines([note + "\n"])

    return "note saved"


note_engine = FunctionTool.from_defaults(
    fn=save_note,
    name="note_saver",
    description="this tool can save a text based note to a file for the user",
)

In [34]:
def Store_Trends(df):
    df['Date'] = pd.to_datetime(df['Date'])
    df['Day'] = df['Date'].dt.day
    df['Month'] = df['Date'].dt.month
    df['Year'] = df['Date'].dt.year
    df.drop(columns=['Date'], inplace=True)
    
    agg_df = df.groupby("Store ID").agg(
        Total_Products=('Product ID', 'count'),
        Total_Sales=('Sales Quantity', 'sum'),
        Avg_Price=('Price', 'mean'),
        Prod_Sold_on_Promotion=('Promotions', lambda x: (x == 'Yes').sum()),
        Products_Festival=('Seasonality Factors', lambda x: (x == 'Festival').sum()),
        Products_Holiday=('Seasonality Factors', lambda x: (x == 'Holiday').sum()),
        Competitor_Affected_Prods=('External Factors', lambda x: (x == 'Competitor Pricing').sum()),
        Weather_Affected_Prods=('External Factors', lambda x: (x == 'Weather').sum()),
        High_Demand_Prods=('Demand Trend', lambda x: (x == 'Increasing').sum()),
        Regular_Customer=('Customer Segments', lambda x: (x == 'Regular').sum()),
        Premium_Customer=('Customer Segments', lambda x: (x == 'Premium').sum()),
        Budget_Customer=('Customer Segments', lambda x: (x == 'Budget').sum()),
        Max_Sale_Day=('Day', lambda x: x.mode()[0] if not x.mode().empty else None),
        Max_Sale_Month=('Month', lambda x: x.mode()[0] if not x.mode().empty else None)).reset_index()
    
    agg_df['Avg_Sales'] = (agg_df['Total_Sales'] / agg_df['Total_Products']).round().astype(int)
    agg_df['Avg_Price'] = agg_df['Avg_Price'].round(2)
    
    return agg_df
    
Store_Trends_engine = FunctionTool.from_defaults(
    fn=Store_Trends,
    name="Store_Trends",
    description="""parameters=[{name: df, type: dataframe, description: Returns:pd.DataFrame}]
                   Returns:pd.DataFrame: An aggregated DataFrame summarizing sales trends across different stores.This tool needs a 
                   Total Products, Average Sales, Average Price of Products, 
                   Number of Products Sold With Promotions, Number of Products Sold in Festival, Number of Products Sold in Holiday,
                   Number of Products affected by Competitors, Number of Products affected by Weather, Number of Products with Higher Demand, 
                   Number of Regular Costumers, Number of Premium Costumers, Number of Budget Costumers, Dat of Maximum Sale, Month of Maximum Sale""",)


In [35]:
from llama_index.core.tools import FunctionTool

def Store_Trends(store_id=None, df=None):
    """
    Analyze store trends from a dataframe.
    
    Args:
        store_id (int, optional): If provided, will filter data for this store.
        df (DataFrame, optional): If provided, will use this dataframe instead of querying.
        
    Returns:
        DataFrame: Aggregated store trends
    """
    # If no dataframe is provided but store_id is, get the data
    if df is None and store_id is not None:
        # You need access to your demand_data function here
        # This is a placeholder for how you might get the data
        query = f"df[df['Store ID'] == {store_id}]"
        df = pd.DataFrame(demand_data(input=query))
    
    if df is None or df.empty:
        return pd.DataFrame({"Error": ["No data available for analysis"]})
    
    # Convert Date to datetime if it exists
    if 'Date' in df.columns:
        df['Date'] = pd.to_datetime(df['Date'])
        df['Day'] = df['Date'].dt.day
        df['Month'] = df['Date'].dt.month
        df['Year'] = df['Date'].dt.year
        df.drop(columns=['Date'], inplace=True)
    
    agg_df = df.groupby("Store ID").agg(
        Total_Products=('Product ID', 'count'),
        Total_Sales=('Sales Quantity', 'sum'),
        Avg_Price=('Price', 'mean'),
        Prod_Sold_on_Promotion=('Promotions', lambda x: (x == 'Yes').sum()),
        Products_Festival=('Seasonality Factors', lambda x: (x == 'Festival').sum()),
        Products_Holiday=('Seasonality Factors', lambda x: (x == 'Holiday').sum()),
        Competitor_Affected_Prods=('External Factors', lambda x: (x == 'Competitor Pricing').sum()),
        Weather_Affected_Prods=('External Factors', lambda x: (x == 'Weather').sum()),
        High_Demand_Prods=('Demand Trend', lambda x: (x == 'Increasing').sum()),
        Regular_Customer=('Customer Segments', lambda x: (x == 'Regular').sum()),
        Premium_Customer=('Customer Segments', lambda x: (x == 'Premium').sum()),
        Budget_Customer=('Customer Segments', lambda x: (x == 'Budget').sum()),
        Max_Sale_Day=('Day', lambda x: x.mode()[0] if not x.mode().empty else None),
        Max_Sale_Month=('Month', lambda x: x.mode()[0] if not x.mode().empty else None)).reset_index()
    
    agg_df['Avg_Sales'] = (agg_df['Total_Sales'] / agg_df['Total_Products']).round().astype(int)
    agg_df['Avg_Price'] = agg_df['Avg_Price'].round(2)
    
    return agg_df

# Create a more flexible tool
Store_Trends_engine = FunctionTool.from_defaults(
    fn=Store_Trends,
    name="Store_Trends",
    description="""
    Analyze store trends from sales data.
    
    Parameters:
    - store_id (int, optional): ID of the store to analyze. If provided, will automatically query data.
    - df (DataFrame, optional): Pre-filtered dataframe to analyze. Use this if you've already queried data.
    
    Returns:
    pd.DataFrame: An aggregated DataFrame summarizing sales trends across different stores, including:
    Total Products, Average Sales, Average Price of Products, 
    Number of Products Sold With Promotions, Number of Products Sold in Festival, Number of Products Sold in Holiday,
    Number of Products affected by Competitors, Number of Products affected by Weather, Number of Products with Higher Demand, 
    Number of Regular Customers, Number of Premium Customers, Number of Budget Customers, Day of Maximum Sale, Month of Maximum Sale.
    """,
)

In [36]:
# Then modify the agent integration to help it work with data
from llama_index.core.tools import QueryEngineTool
from llama_index.core.memory import ChatMemoryBuffer

# Create a memory buffer to store the dataframe
memory = ChatMemoryBuffer.from_defaults(token_limit=1500)

# Create a special tool to help with data flow
def store_data_helper(store_id=None):
    """
    Helper function to get store data and prepare it for analysis
    """
    # Get the data
    query = f"df[df['Store ID'] == {store_id}]"
    data = demand_query_engine.query(query)
    
    # Convert to DataFrame if needed (depends on your query_engine return type)
    df = pd.DataFrame(data.response)
    
    # Store in memory for other tools to access
    memory.put("store_data", df)
    
    # Return a summary
    return f"Retrieved data for Store {store_id} with {len(df)} records. Use Store_Trends tool to analyze this data."

# Create the data helper tool
data_helper_tool = FunctionTool.from_defaults(
    fn=store_data_helper,
    name="get_store_data",
    description="Get data for a specific store ID and prepare it for analysis with Store_Trends"
)

In [37]:
# Update your tools list
tools = [
    Store_Trends_engine,
    data_helper_tool,
    note_engine,
    QueryEngineTool(
        query_engine=demand_query_engine,
        metadata=ToolMetadata(
            name="demand_data",
            description="""This dataset contains sales transaction data with Product ID,
            Date, Store ID, Sales Quantity, Price, Promotions, Seasonality Factors, External Factors, Demand Trend,
            and Customer Segments.""",
        ),
    ),
]

In [38]:
agent = ReActAgent.from_tools(tools, llm=llm, verbose=True, context=context, memory=memory)

while (prompt := input("Enter a prompt (q to quit): ")) != "q":
    result = agent.query(prompt)

    print(result)

ReActChatFormatter.from_context is deprecated, please use `from_defaults` instead.


Enter a prompt (q to quit):  store trends of store 48


> Running step 52c5eb50-f09b-419a-a666-41cab9b27b65. Step input: store trends of store 48
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: get_store_data
Action Input: {'store_id': 48}
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
> Pandas Instructions:
```
df.loc[df['Store ID'] == 48]
```
> Pandas Output:       Product ID        Date  Store ID  Sales Quantity  Price Promotions  \
0           4277  2024-01-03        48             330  24.38         No   
149         7227  2024-10-26        48             250  43.43         No   
284         2240  2024-02-02        48             386  83.13         No   
286  

ValueError: Reached max iterations.

In [39]:
from llama_index.core.tools import FunctionTool
from llama_index.core.agent import ReActAgent
import pandas as pd

# Create a global state dictionary to store dataframes
agent_state = {
    "dataframes": {}
}

# Function to save a dataframe to the state
def save_dataframe(df, name="filtered_df"):
    """Save a dataframe to the agent state with a given name"""
    agent_state["dataframes"][name] = df
    return f"Dataframe saved as '{name}'"

# Function to get a dataframe from the state
def get_dataframe(name="filtered_df"):
    """Retrieve a dataframe from the agent state by name"""
    if name in agent_state["dataframes"]:
        return agent_state["dataframes"][name]
    else:
        return None

# Modified demand_data function that saves the result
def demand_data_with_save(input):
    """Query the demand data and save the result as a dataframe"""
    result = demand_query_engine.query(input)
    
    # Convert the result to a dataframe (adjust based on your actual return type)
    # This depends on how your demand_query_engine returns data
    df = pd.DataFrame(result.response)  # Adjust this line as needed
    
    # Save the dataframe to state
    save_dataframe(df, "current_df")
    
    return f"Data filtered with query '{input}' and saved as 'current_df'. Use Store_Trends to analyze this data."

# Modified Store_Trends function that works with the state
def Store_Trends(df_name="current_df"):
    """
    Analyze store trends from a dataframe stored in the agent state.
    
    Args:
        df_name (str): Name of the dataframe in the agent state
        
    Returns:
        DataFrame: Aggregated store trends
    """
    # Get the dataframe from state
    df = get_dataframe(df_name)
    
    if df is None or df.empty:
        return pd.DataFrame({"Error": ["No data available for analysis or dataframe not found"]})
    
    # Convert Date to datetime if it exists
    if 'Date' in df.columns:
        df['Date'] = pd.to_datetime(df['Date'])
        df['Day'] = df['Date'].dt.day
        df['Month'] = df['Date'].dt.month
        df['Year'] = df['Date'].dt.year
        df.drop(columns=['Date'], inplace=True)
    
    agg_df = df.groupby("Store ID").agg(
        Total_Products=('Product ID', 'count'),
        Total_Sales=('Sales Quantity', 'sum'),
        Avg_Price=('Price', 'mean'),
        Prod_Sold_on_Promotion=('Promotions', lambda x: (x == 'Yes').sum()),
        Products_Festival=('Seasonality Factors', lambda x: (x == 'Festival').sum()),
        Products_Holiday=('Seasonality Factors', lambda x: (x == 'Holiday').sum()),
        Competitor_Affected_Prods=('External Factors', lambda x: (x == 'Competitor Pricing').sum()),
        Weather_Affected_Prods=('External Factors', lambda x: (x == 'Weather').sum()),
        High_Demand_Prods=('Demand Trend', lambda x: (x == 'Increasing').sum()),
        Regular_Customer=('Customer Segments', lambda x: (x == 'Regular').sum()),
        Premium_Customer=('Customer Segments', lambda x: (x == 'Premium').sum()),
        Budget_Customer=('Customer Segments', lambda x: (x == 'Budget').sum()),
        Max_Sale_Day=('Day', lambda x: x.mode()[0] if not x.mode().empty else None),
        Max_Sale_Month=('Month', lambda x: x.mode()[0] if not x.mode().empty else None)).reset_index()
    
    agg_df['Avg_Sales'] = (agg_df['Total_Sales'] / agg_df['Total_Products']).round().astype(int)
    agg_df['Avg_Price'] = agg_df['Avg_Price'].round(2)
    
    # Save the result too for potential future use
    save_dataframe(agg_df, "trends_df")
    
    return agg_df

# Create the tools with the new approach
demand_data_tool = FunctionTool.from_defaults(
    fn=demand_data_with_save,
    name="demand_data",
    description="""
    Query the sales transaction data and save the result as a dataframe.
    
    Parameters:
    - input (str): A query string, typically to filter data (e.g., "df[df['Store ID'] == 48]")
    
    Returns:
    str: A confirmation message that the data has been filtered and saved
    
    Note: This tool saves the filtered data with the name 'current_df' for use with other tools.
    """
)

store_trends_tool = FunctionTool.from_defaults(
    fn=Store_Trends,
    name="Store_Trends",
    description="""
    Analyze store trends from a saved dataframe.
    
    Parameters:
    - df_name (str, optional): Name of the saved dataframe to analyze. Defaults to 'current_df'.
    
    Returns:
    pd.DataFrame: An aggregated DataFrame summarizing sales trends.
    
    Note: This tool expects that data has already been queried and saved using the demand_data tool.
    """
)

# Update your tools list
tools = [
    store_trends_tool,
    demand_data_tool,
    note_engine,
]

# Create the agent
agent = ReActAgent.from_tools(tools, llm=llm, verbose=True, context=context)

while (prompt := input("Enter a prompt (q to quit): ")) != "q":
    result = agent.query(prompt)

    print(result)

ReActChatFormatter.from_context is deprecated, please use `from_defaults` instead.


Enter a prompt (q to quit):  store trends of store 48


> Running step 39509bec-cd93-4615-967c-70d67901ad2e. Step input: store trends of store 48
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: demand_data
Action Input: {'input': "df[df['Store ID'] == 48]"}
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
> Pandas Instructions:
```
df.loc[df['Store ID'] == 48]
```
> Pandas Output:       Product ID        Date  Store ID  Sales Quantity  Price Promotions  \
0           4277  2024-01-03        48             330  24.38         No   
149         7227  2024-10-26        48             250  43.43         No   
284         2240  2024-02-02        48             386  83.13  

ValueError: Reached max iterations.